In [0]:
%run /Workspace/Repos/dataeng.victor@outlook.com/databricks-weather-realtime/02.bronze/01_bronze_autoloader


In [0]:
%python
from pyspark.sql.functions import col

# ler arquivos text criados pelo ingest_api
df = (
    spark.readStream.format("cloudFiles")
        .option("cloudFiles.format", "text")
        .option("cloudFiles.schemaLocation", bronze["schema_location"])
        .load(bronze["input"])
)

# transformar a coluna de texto em JSON bruto
df_json = df.select(col("value").alias("json_str"))



In [0]:
%python
# gravar tabela Delta no Bronze
(
    df_json.writeStream
        .format("delta")
        .option("checkpointLocation", bronze["checkpoint"])
        .trigger(availableNow=True)  
        .table(f"{catalog}.{bronze['schema']}.{bronze['table']}")
)

In [0]:
%sql
USE CATALOG weather_realtime

In [0]:
%sql
SELECT * FROM bronze.weather_raw